# FEB3 — Notebook 1 (ETL)  
## Connexió a MongoDB (views) → DataFrames (pandas)


### Vistes utilitzades
- `vw_FEB3_players_base_3y` → 1 fila per jugador i temporada (features agregades)
- `vw_FEB3_shots_clean_3y` → tirs nets (coordenades, zona, tipus, encert)




## 1) Connexió a MongoDB


In [30]:
from pymongo import MongoClient


MONGO_URI = "mongodb://10.98.254.189:27017/"
DB_NAME = "feb_db"

client = MongoClient(MONGO_URI)
db = client[DB_NAME]

print("Connectat a MongoDB:", MONGO_URI, "| DB:", DB_NAME)


Connectat a MongoDB: mongodb://10.98.254.189:27017/ | DB: feb_db


Aquest punt estableix la connexió amb el servidor MongoDB mitjançant la llibreria MongoClient.
Un cop connectat, es selecciona la base de dades feb_db, que conté les col·leccions i vistes utilitzades en el projecte.

## 2) Càrrega del dataset base (jugadors) des de la vista


In [29]:
import pandas as pd

VIEW_PLAYERS = "vw_FEB3_players_base_3y"

cursor_players = db[VIEW_PLAYERS].find()
df_players = pd.DataFrame(list(cursor_players))

print("Vista carregada:", VIEW_PLAYERS)
print("Files, columnes:", df_players.shape)
df_players.head()


Vista carregada: vw_FEB3_players_base_3y
Files, columnes: (4755, 13)


,games,avg_min,avg_pts,avg_ast,avg_trb,avg_tov,avg_2pa,avg_3pa,fg2_pct,fg3_pct,avg_eff,player_feb_id,season_id
0,12,24.131944,13.583333,1.000000,6.166667,1.833333,6.250000,5.250000,0.388757,0.387037,12.250000,2816298,2025
1,26,33.455128,17.846154,1.500000,8.153846,3.000000,10.961538,0.961538,0.607855,0.186275,21.653846,2648564,2023-2024
2,12,27.731944,7.416667,3.833333,5.250000,2.833333,4.333333,1.500000,0.395238,0.435185,12.500000,1384586,2025
3,23,23.016667,7.130435,2.739130,1.869565,2.478261,2.478261,3.521739,0.417293,0.322826,6.086957,1246691,2023-2024
4,13,26.937179,8.000000,1.538462,6.538462,2.692308,9.384615,0.384615,0.360577,0.000000,6.769231,1451287,2023-2024


En aquest pas es carrega el dataset base de jugadors a partir de la vista vw_FEB3_players_base_3y, que ja conté les dades agregades per jugador i temporada.
Les dades obtingudes es converteixen en un DataFrame de pandas per facilitar la seva anàlisi i despres el  tractament.

## 3) Observació inicial (qualitat i tipus de dades)


In [31]:
df_players.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4755 entries, 0 to 4754
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   games          4755 non-null   int64  
 1   avg_min        4755 non-null   float64
 2   avg_pts        4755 non-null   float64
 3   avg_ast        4755 non-null   float64
 4   avg_trb        4755 non-null   float64
 5   avg_tov        4755 non-null   float64
 6   avg_2pa        4755 non-null   float64
 7   avg_3pa        4755 non-null   float64
 8   fg2_pct        4750 non-null   float64
 9   fg3_pct        4586 non-null   float64
 10  avg_eff        4755 non-null   float64
 11  player_feb_id  4755 non-null   object 
 12  season_id      4755 non-null   object 
dtypes: float64(10), int64(1), object(2)
memory usage: 483.1+ KB


Aquesta secció permet analitzar l’estructura del dataset, mostrant el tipus de dades i el nombre de valors no nuls per a cada columna.


## 4) Descripció estadística


In [32]:
df_players.describe(include="all")


,games,avg_min,avg_pts,avg_ast,avg_trb,avg_tov,avg_2pa,avg_3pa,fg2_pct,fg3_pct,avg_eff,player_feb_id,season_id
count,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000,4755.000000,4750.000000,4586.000000,4755.000000,4755,4755
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2921,3
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1074329,2023-2024
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1701
mean,17.613039,19.319419,7.281652,1.361977,3.596370,1.458678,3.879278,2.516181,0.458368,0.243968,7.238863,NaN,NaN
std,6.338177,6.821364,4.315119,1.045003,2.206159,0.783827,2.450018,1.991453,0.124860,0.127384,5.307654,NaN,NaN
min,8.000000,2.017708,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.600000,NaN,NaN
25%,12.000000,14.621576,3.909091,0.611111,1.956522,0.880000,2.000000,0.881176,0.389568,0.174702,3.078462,NaN,NaN
50%,18.000000,19.679545,6.800000,1.090909,3.166667,1.363636,3.520000,2.117647,0.464818,0.253643,6.416667,NaN,NaN
75%,24.000000,24.409778,10.083333,1.840000,4.815341,1.956522,5.382784,3.800000,0.533333,0.319409,10.434783,NaN,NaN


Aquesta anàlisi proporciona un resum estadístic de totes les variables del dataset, incloent mesures com la mitjana, la desviació estàndard i els valors mínim i màxim.
Permet identificar la distribució de les dades i detectar possibles valors extrems abans de realitzar una anàlisi exploratòria més detallada.

## 5) Selecció de variables numèriques


In [33]:
numeric_df = df_players.select_dtypes(include="number").copy()

print("Columnes numèriques:", list(numeric_df.columns))
numeric_df.head()


Columnes numèriques: ['games', 'avg_min', 'avg_pts', 'avg_ast', 'avg_trb', 'avg_tov', 'avg_2pa', 'avg_3pa', 'fg2_pct', 'fg3_pct', 'avg_eff']


,games,avg_min,avg_pts,avg_ast,avg_trb,avg_tov,avg_2pa,avg_3pa,fg2_pct,fg3_pct,avg_eff
0,12,24.131944,13.583333,1.000000,6.166667,1.833333,6.250000,5.250000,0.388757,0.387037,12.250000
1,26,33.455128,17.846154,1.500000,8.153846,3.000000,10.961538,0.961538,0.607855,0.186275,21.653846
2,12,27.731944,7.416667,3.833333,5.250000,2.833333,4.333333,1.500000,0.395238,0.435185,12.500000
3,23,23.016667,7.130435,2.739130,1.869565,2.478261,2.478261,3.521739,0.417293,0.322826,6.086957
4,13,26.937179,8.000000,1.538462,6.538462,2.692308,9.384615,0.384615,0.360577,0.000000,6.769231


En aquest pas es filtren únicament les variables numèriques del dataset, ja que són les úniques adequades per a tècniques de clustering.


## 6) Tractament de valors nuls


In [34]:
nulls = numeric_df.isna().sum().sort_values(ascending=False)
nulls[nulls > 0].head(20)


fg3_pct    169
fg2_pct      5
dtype: int64

In [35]:
# - Els nuls solen venir de percentatges quan no hi ha intents).
# - Els convertim a 0 per evitar problemes d'escalat/model.
numeric_df.fillna(0, inplace=True)

print("Nuls després del tractament:", int(numeric_df.isna().sum().sum()))

Nuls després del tractament: 0


En aquest pas s’identifiquen les columnes amb valors nuls per analitzar-ne l’origen i la seva concentració.
Els valors absents es substitueixen per zero, ja que en aquest context solen correspondre a percentatges sense intents, evitant així problemes en l’escalat i els models posteriors.

## 7) Càrrega del dataset de tirs des de la vista


In [36]:
VIEW_SHOTS = "vw_FEB3_shots_clean_3y"

cursor_shots = db[VIEW_SHOTS].find()
df_shots = pd.DataFrame(list(cursor_shots))

print("Vista carregada:", VIEW_SHOTS)
print("Files, columnes:", df_shots.shape)
df_shots.head()


Vista carregada: vw_FEB3_shots_clean_3y
Files, columnes: (565311, 12)


,player_feb_id,team_feb_id,x,y,court_region,season_id,match_feb_id,data,hora,type,made,competition_name
0,2643006,979921,83.152904,47.280680,PC,2025,2487031,04-10-2025,18:00,2p,False,LIGA EBA
1,1749166,979921,80.378887,49.699131,PC,2025,2487031,04-10-2025,18:00,2p,False,LIGA EBA
2,2807682,981221,88.092013,51.754815,PC,2025,2487031,04-10-2025,18:00,2p,False,LIGA EBA
3,1749166,979921,66.373475,51.392048,Ce3R,2025,2487031,04-10-2025,18:00,3p,False,LIGA EBA
4,2807682,981221,10.148849,48.248062,PC,2025,2487031,04-10-2025,18:00,2p,True,LIGA EBA


En aquest pas es carrega el dataset de tirs a partir de la vista vw_FEB3_shots_clean_3y, que conté les dades de tir ja netejades.
Aquest conjunt de dades permet analitzar patrons d’encert segons zona de pista, tipus de tir i temporada dins l’EDA.

## 8) Validacions ràpides dels tirs


In [37]:
# Percentatge global d'encert i volum per tipus
if not df_shots.empty:
    print("Encert global:", df_shots["made"].mean())
    print("Volum per tipus:")
    display(df_shots["type"].value_counts())
else:
    print(" df_shots està buit.")


Encert global: 0.40790821335512667
Volum per tipus:


type
2p    340501
3p    224810
Name: count, dtype: int64

En aquesta secció es comprova ràpidament que el dataset de tirs conté dades correctes. El percentatge global d’encert és d’aproximadament un 40%, un valor coherent per aquest tipus de competició.

També es mostra el volum de tirs per tipus, on es veu que hi ha més tirs de 2 punts que de 3 punts, fet que concorda amb el joc real. Aquestes comprovacions serveixen per validar les dades abans de fer anàlisis més detallades.

In [39]:


# Comprovem que el dataset de tirs no estigui buit
if df_shots.empty:
    print("df_shots està buit.")
else:
    # % d'encert global
    encert_global = df_shots["made"].mean()
    print("Encert global:", round(encert_global, 4))

    # Volum de tirs per tipus (2p / 3p)
    print("\nVolum per tipus:")
    display(df_shots["type"].value_counts())

    # % d'encert per tipus
    print("\nEncert per tipus:")
    display(df_shots.groupby("type")["made"].mean().sort_values(ascending=False))

    # Volum per zona de pista
    print("\nVolum per zona (court_region):")
    display(df_shots["court_region"].value_counts().head(15))

    # Encerc per zona (només si hi ha la columna court_region)
    if "court_region" in df_shots.columns:
        print("\nEncert per zona (top 15):")
        display(df_shots.groupby("court_region")["made"].mean().sort_values(ascending=False).head(15))


Encert global: 0.4079

Volum per tipus:


type
2p    340501
3p    224810
Name: count, dtype: int64


Encert per tipus:


type
2p    0.476844
3p    0.303496
Name: made, dtype: float64


Volum per zona (court_region):


court_region
PC      120175
Ce3L     78638
PR       78504
Ce3R     76192
PL       70892
E3L      24907
E3R      21199
MBR      18381
MER      18175
MBL      17592
MEL      16782
C3L       8753
C3R       7446
Name: count, dtype: int64


Encert per zona (top 15):


court_region
PC      0.573996
PR      0.478065
PL      0.460249
MEL     0.330533
MER     0.329849
MBR     0.328927
C3L     0.323889
MBL     0.320600
C3R     0.319769
E3L     0.306139
E3R     0.304212
Ce3R    0.297210
Ce3L    0.294120
Name: made, dtype: float64

En aquest apartat hem comprovat que el dataset de tirs no està buit i que les dades són coherents. Hem vist que els tirs de 2 punts són més freqüents i tenen més encert que els de 3 punts, i que hi ha diferències segons la zona del camp.

## 9) Detecció i justificació d’outliers

In [40]:
import numpy as np

# Ens quedem només amb les variables numèriques del dataset de jugadors
df_num = df_players.select_dtypes(include="number").copy()

# Càlcul del rang interquartílic (IQR)
Q1 = df_num.quantile(0.25)
Q3 = df_num.quantile(0.75)
IQR = Q3 - Q1

# Definim els límits inferior i superior
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Comptem quants outliers hi ha per variable
outliers_per_variable = (
    (df_num < lower_bound) | (df_num > upper_bound)
).sum().sort_values(ascending=False)

outliers_per_variable


avg_ast    174
fg2_pct    163
avg_trb    107
avg_tov     64
avg_eff     63
avg_2pa     53
fg3_pct     46
avg_pts     40
avg_3pa     39
avg_min      0
games        0
dtype: int64

En aquest apartat hem detectat possibles outliers utilitzant el mètode del rang interquartílic (IQR) sobre les variables numèriques dels jugadors. S’ha observat que algunes variables com les assistències, percentatge de tir i rebots presenten més valors extrems.

Aquests outliers poden correspondre a jugadors amb rendiments molt alts o situacions puntuals, per la qual cosa es tenen en compte però no s’eliminen automàticament.

## 10) Eliminació de variables redundants

In [41]:
import numpy as np

# Matriu de correlació absoluta
corr_matrix = df_num.corr().abs()

# Ens quedem només amb el triangle superior per evitar duplicats
upper_triangle = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

# Llindar de correlació alta
threshold = 0.85

# Identifiquem variables redundants
redundant_features = [
    col for col in upper_triangle.columns
    if any(upper_triangle[col] > threshold)
]

redundant_features


['avg_pts', 'avg_eff']

In [42]:
# Eliminem les variables redundants
df_num_reduced = df_num.drop(columns=redundant_features)

df_num_reduced.shape


(4755, 9)

En aquest apartat hem analitzat la correlació entre les variables numèriques per detectar informació redundant. Les variables amb una correlació alta (superior a 0,85), com avg_pts i avg_eff, s’han considerat redundants i s’han eliminat.

Això ens permet reduir la dimensionalitat i treballar amb un dataset més compacte per al clustering.

## 11) Justificació de les features escollides

In [43]:
features_final = df_num_reduced.columns.tolist()
features_final

['games',
 'avg_min',
 'avg_ast',
 'avg_trb',
 'avg_tov',
 'avg_2pa',
 'avg_3pa',
 'fg2_pct',
 'fg3_pct']

Hem seleccionat les features a partir del dataset ja net i reduït. Aquestes variables permeten representar el volum de joc, la producció ofensiva i l’eficiència dels jugadors.

Aquesta selecció és suficient per descriure diferents perfils de jugadors i aplicar el clustering sense afegir informació redundant.

## 12) Exportació del dataset final per al Playbook 2

In [44]:
# Columnes identificadores que volem conservar (si existeixen)
id_cols = [col for col in ["player_feb_id", "season_id"] if col in df_players.columns]

# Construïm el dataset final combinant identificadors i features numèriques finals
df_final = pd.concat(
    [
        df_players[id_cols].reset_index(drop=True),
        df_num_reduced.reset_index(drop=True)
    ],
    axis=1
)

# Comprovem dimensions
df_final.shape

(4755, 11)

In [45]:
# Exportem a CSV
df_final.to_csv("FEB3_Playbook2_ready.csv", index=False)


En aquest apartat hem construït el dataset final combinant els identificadors del jugador i la temporada amb les features numèriques seleccionades. Hem comprovat que les dimensions són correctes i que el conjunt està preparat per al següent pas.

Finalment, hem exportat el dataset en format CSV per poder utilitzar-lo al Playbook 2, on s’aplicarà el clustering.

## Conclusions finals

Hem filtrat i netejat les dades, creat vistes específiques i construït un dataset final coherent i preparat per a tècniques de Machine Learning.

Durant el procés hem validat les dades, detectat possibles outliers i eliminat variables redundants per reduir la dimensionalitat. També hem seleccionat les features més rellevants per representar el rendiment dels jugadors de manera equilibrada.

Finalment, hem exportat el dataset final en format CSV per utilitzar-lo en el següent playbook, on s’aplicaran tècniques de clustering no supervisat amb l’objectiu de descobrir diferents perfils de jugadors.